<a href="https://colab.research.google.com/github/aimbsg/EIP4-Phase1-Final-Assignment/blob/master/EIP4_Phase_1_Final_Assignment_PersonAttributes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
import os
from keras.layers import GlobalAveragePooling2D
from keras.models import load_model
from keras.optimizers import Adadelta
from keras.utils import plot_model

from keras.backend import sigmoid
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation

In [0]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
# remove unwanted columns
del df["filename"] 
#del df["image_path"]
df.head()

In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True, augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation = augmentation

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        if self.augmentation is not None:
          image = self.augmentation.flow(image, shuffle=False).next()
        
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [0]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15, random_state = 1234)
train_df.shape, val_df.shape

In [0]:
train_df.head()

In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32, augmentation = ImageDataGenerator(rescale=1./255,
        #featurewise_center = True,
        #featurewise_std_normalization =True,
        shear_range=0.2,
        zoom_range=0.4,
        #brightness_range=[0.5,1.5],
        horizontal_flip = True,
        vertical_flip=True,
        zca_whitening = True#,
        #fill_mode='nearest'))
# 0-80 epochs shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True
# 80-120 epochs shear_range = 0.1, zoom_range = 0.1, vertical_flip = True
# 120-160 epochs shear_range = 0.2, zoom_range = 0.4, vertical_flip = True, zca_whitening = True
valid_gen = PersonDataGenerator(val_df, batch_size=32, shuffle=False, augmentation=ImageDataGenerator(rescale=1./255))

In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

In [0]:
# Defining swish() function
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
  
class Swish(Activation):
    
    def __init__(self, activation, **kwargs):
        super(Swish, self).__init__(activation, **kwargs)
        self.__name__ = 'swish'

def swish(x):
    return (K.sigmoid(x) * x)

get_custom_objects().update({'swish': Swish(swish)})

In [0]:
#backbone = ResNet50(
#    weights=None, 
#    include_top=False, 
#    input_tensor=Input(shape=(224, 224, 3))
#)

backbone = InceptionResNetV2(
    weights=None, 
    include_top=False, 
    input_tensor=Input(shape=(224, 224, 3))
    #,pooling='max'
)

neck = backbone.output
#neck = GlobalAveragePooling2D()(neck)
neck = Flatten(name="flatten")(neck)
neck = Dense(256, activation="relu")(neck) 


def build_tower(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    #neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)    
    return neck


def build_head(name, act_func, in_layer):
    return Dense(
        num_units[name], activation=act_func, name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", "softmax", build_tower(neck))
image_quality = build_head("image_quality", "softmax", build_tower(neck))
age = build_head("age", "swish", build_tower(neck))
weight = build_head("weight", "relu", build_tower(neck))
bag = build_head("bag", "sigmoid", build_tower(neck))
footwear = build_head("footwear", "sigmoid", build_tower(neck))
emotion = build_head("emotion", "sigmoid", build_tower(neck))
pose = build_head("pose", "sigmoid", build_tower(neck))

#model = ResNet50(include_top=False, weights=None, input_tensor=None, input_shape=Input(shape=(224, 224, 3)), pooling=None, classes=1000)

In [0]:
model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

In [0]:
# freeze backbone
for layer in backbone.layers:
	layer.trainable = True

In [0]:
 losses = {
 	"gender_output": "binary_crossentropy",
 	"image_quality_output": "categorical_crossentropy",
 	"age_output": "categorical_crossentropy",
 	"weight_output": "categorical_crossentropy",
   "bag_output": "categorical_crossentropy",
   "footwear_output": "categorical_crossentropy",
   "pose_output": "categorical_crossentropy",
   "emotion_output": "categorical_crossentropy"
 }
# loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0}
sgd = SGD(lr=0.0001, momentum=0.85, nesterov=False)
model.compile(
    optimizer=sgd,
    #optimizer = Adadelta(lr=1.0, rho=0.95),
    #optimizer=Adam(lr=lr_schedule(0)),
    loss=losses, 
    # loss_weights=loss_weights, 
    metrics=["accuracy"]
)
model.summary()

In [0]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 0.001  # 1e-3 was default till 40 epochs; 1e-4 till 80 epochs; 1e-5 till 160 epochs
    if epoch > 80:
        lr *= 0.00001
    elif epoch > 40:
        lr *= 0.0001
    print('Learning rate: ', lr)
    return lr

In [0]:
# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'Inc_ResNetModel.{epoch:03d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_age_output_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

#callbacks = [checkpoint, lr_reducer, lr_scheduler]
callbacks = [checkpoint, lr_scheduler]

In [0]:
# load model

model = load_model('/content/gdrive/My Drive/Qual_Assign_Inc_Resnet_model_120epochs.h5')

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=1, 
    epochs=160,
    verbose=2,
    callbacks=callbacks,
    initial_epoch=120
)

In [0]:
val_results = model.evaluate_generator(valid_gen, verbose=1)
dict(zip(model.metrics_names, val_results))

In [0]:
model.save('/content/gdrive/My Drive/Qual_Assign_Inc_Resnet_model_160epochs.h5', overwrite=True)